In [1]:
### load datasets

In [2]:
import torch

In [3]:
# Install Hugging Face datasets library

from datasets import load_dataset

# Load version 3.0.0 of the CNN/DailyMail dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")


d:\TextSummarize\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Access a sample article and its summary
sample = dataset["test"][0]
print("📰 Article:\n", sample["article"])
print("\n📌 Reference Summary:\n", sample["highlights"])


📰 Article:
 (CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday's 

In [5]:
#### EXTRACTIVE SUMMARIZATION WITH SUMY

In [6]:


import nltk
nltk.download('punkt')

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

def extractive_summary(text, num_sentences=3):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LexRankSummarizer()
    summary = summarizer(parser.document, num_sentences)
    return " ".join(str(sentence) for sentence in summary)

print("\n🔹 Extractive Summary:\n", extractive_summary(sample["article"]))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!



🔹 Extractive Summary:
 "As the Rome Statute today enters into force for the State of Palestine, Palestine acquires all the rights as well as responsibilities that come with being a State Party to the Statute. "What's objectionable is the attempts to undermine international justice, not Palestine's decision to join a treaty to which over 100 countries around the world are members." "As we have said repeatedly, we do not believe that Palestine is a state and therefore we do not believe that it is eligible to join the ICC," the State Department said in a statement.


In [7]:
### ABRSTRACTIVE SUMMARIZATION WITH TRANSFORMERS

In [8]:
import torch
from transformers import pipeline
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
# ...existing code..."facebook/bart-large-cnn")

def abstractive_summary(text):
    summary = summarizer(text, max_length=130, min_length=30, do_sample=False)
    return summary[0]['summary_text']

Device set to use cpu


In [9]:
### EVALUATION WITH ROUGE
from rouge_score import rouge_scorer

reference_summary = sample["highlights"]
generated_summary = extractive_summary(sample["article"])  # or abstractive_summary(sample["article"])

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference_summary, generated_summary)
print(scores)

{'rouge1': Score(precision=0.11578947368421053, recall=0.3235294117647059, fmeasure=0.17054263565891473), 'rouge2': Score(precision=0.010638297872340425, recall=0.030303030303030304, fmeasure=0.015748031496062992), 'rougeL': Score(precision=0.07368421052631578, recall=0.20588235294117646, fmeasure=0.10852713178294572)}
